# Product Recommendation with Feathr

This notebook illustrates the use of Feathr Feature Store to create a model that predict users' rating for different products for a e-commerce website.

### Model Problem Statement
The e-commerce website has collected past user ratings for various products. The website also collected data about user and product, like user age, product category etc. Now we want to predict users' product rating for new product so that we can recommend the new product to users that give a high rating for those products.

### Feature Creation Illustration
In this example, our observation data has compound entity key where a record is uniquely identified by `user_id` and `product_id`. With that, we can think about three types of features:
1. **User features** that are different for different users but are the same for different products. For example, user age is different for different users but it's product-agnostic.
2. **Product features** that are different for different products but are the same for all the users.
3. **User-to-product** features that are different for different users AND different products. For example, a feature to represent if the user has bought this product before or not.

In this example, we will focus on the first two types of features. After we train a model based on those features, we predict the product ratings that users will give for the products.

The feature creation flow is as below:
![Feature Flow](https://github.com/feathr-ai/feathr/blob/main/docs/images/product_recommendation_advanced.jpg?raw=true)

## 2. Config Feathr Client

In [1]:
import glob
import os
import tempfile
from datetime import datetime, timedelta
from math import sqrt

import pandas as pd
from pyspark.sql import DataFrame
# from interpret.provider import InlineProvider
# from interpret import set_visualize_provider

# set_visualize_provider(InlineProvider())

import feathr
from feathr import (
    FeathrClient,
    BOOLEAN, FLOAT, INT32, ValueType,
    Feature, DerivedFeature, FeatureAnchor,
    BackfillTime, MaterializationSettings,
    FeatureQuery, ObservationSettings,
    RedisSink,
    INPUT_CONTEXT, HdfsSource,
    WindowAggTransformation,
    TypedKey,
)
from feathr.datasets.constants import (
    PRODUCT_RECOMMENDATION_USER_OBSERVATION_URL,
    PRODUCT_RECOMMENDATION_USER_PROFILE_URL,
    PRODUCT_RECOMMENDATION_USER_PURCHASE_HISTORY_URL,
    PRODUCT_RECOMMENDATION_PRODUCT_DETAIL_URL,
)
from feathr.datasets.utils import maybe_download
from feathr.utils.config import generate_config
from feathr.utils.job_utils import get_result_df

print(f"Feathr version: {feathr.__version__}")

Feathr version: 1.0.0


In [2]:
os.environ['SPARK_LOCAL_IP'] = "127.0.0.1"
os.environ['REDIS_PASSWORD'] = "foobared"
PROJECT_NAME = "product_recommendation_from_jdbc"

### Initialize Feathr Client

In [3]:
from pathlib import Path
feathr_workspace_folder = Path(f"./{PROJECT_NAME}_feathr_config.yaml")
client = FeathrClient(str(feathr_workspace_folder))

2024-09-09 16:16:46.719 | INFO     | feathr.utils._env_config_reader:get:62 - Config secrets__azure_key_vault__name is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-09 16:16:46.721 | INFO     | feathr.utils._env_config_reader:get:62 - Config offline_store__s3__s3_enabled is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-09 16:16:46.721 | INFO     | feathr.utils._env_config_reader:get:62 - Config offline_store__adls__adls_enabled is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-09 16:16:46.722 | INFO     | feathr.utils._env_config_reader:get:62 - Config offline_store__wasb__wasb_enabled is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-09 16:16:46

### What's a feature in Feathr
A feature is an individual measurable property or characteristic of a phenomenon which is sometimes time-sensitive.

In Feathr, a feature is defined by the following characteristics:
* The typed key (a.k.a. entity id): identifies the subject of feature, e.g. a user id of 123, a product id of SKU234456.
* The feature name: the unique identifier of the feature, e.g. user_age, total_spending_in_30_days.
* The feature value: the actual value of that aspect at a particular time, e.g. the feature value of the person's age is 30 at year 2022.
* The timestamp: this indicates when the event happened. For example, the user purchased certain product on a certain timestamp. This is usually used for point-in-time join.

You can feel that this is defined from a feature consumer (a person who wants to use a feature) perspective. It only tells us what a feature is like. In later sections, you can see how a feature consumer can access the features in a very simple way.

To define how to produce the feature, we need to specify:
* Feature source: what source data that this feature is based on
* Transformation: what transformation is used to transform the source data into feature. Transformation can be optional when you just want to take a column out from the source data.

(For more details on feature definition, please refer to the [Feathr Feature Definition Guide](https://feathr-ai.github.io/feathr/concepts/feature-definition.html).)

Note: in some cases, such as features defined on top of request data, may have no entity key or timestamp.
It is merely a function/transformation executing against request data at runtime.
For example, the day of week of the request, which is calculated by converting the request UNIX timestamp.
(We won't cover this in the tutorial.)

### Define Sources Section with UDFs

A feature is called an anchored feature when the feature is directly extracted from the source data, rather than computed on top of other features. The latter case is called derived feature.

A [feature source](https://feathr.readthedocs.io/en/latest/#feathr.Source) is needed for anchored features that describes the raw data in which the feature values are computed from. See the python documentation to get the details on each input column.

See [the python API documentation](https://feathr.readthedocs.io/en/latest/#feathr.HdfsSource) to get the details of each input fields.

### Define window aggregation features

[Window aggregation](https://en.wikipedia.org/wiki/Window_function_%28SQL%29) helps us to create more powerful features by compressing large amount of information. For example, we can compute *average purchase amount over the last 90 days* from the purchase history to capture user's recent consumption trend.

To create window aggregation features, we define `WindowAggTransformation` with following arguments:
1. `agg_expr`: the field/column you want to aggregate. It can be an ANSI SQL expression, e.g. `cast_float(purchase_amount)` to cast `str` type values to `float`.
2. `agg_func`: the aggregation function, e.g. `AVG`. See below table for the full list of supported functions.
3. `window`: the aggregation window size, e.g. `90d` to aggregate over the 90 days.

| Aggregation Type | Input Type | Description |
| --- | --- | --- |
| `SUM`, `COUNT`, `MAX`, `MIN`, `AVG` | Numeric | Applies the the numerical operation on the numeric inputs. |
| `MAX_POOLING`, `MIN_POOLING`, `AVG_POOLING`	| Numeric Vector | Applies the max/min/avg operation on a per entry basis for a given a collection of numbers. |
| `LATEST` | Any | Returns the latest not-null values from within the defined time window. |

After you have defined features and sources, bring them together to build an anchor:

> Note that if the features comes directly from the observation data, the `source` argument should be `INPUT_CONTEXT` to indicate the source of the anchor is the observation data.

## Get features from Register Server

In [4]:
client.list_registered_features(project_name=PROJECT_NAME)

[{'name': 'feature_user_avg_purchase_for_90days',
  'id': 'f4ea07a4-8cf5-4667-8b2c-157ae196154f',
  'qualifiedName': 'product_recommendation_from_jdbc__aggregationFeatures__feature_user_avg_purchase_for_90days'},
 {'name': 'feature_user_age',
  'id': '3dc6a53e-6926-4ab5-9f45-b88afb1be57c',
  'qualifiedName': 'product_recommendation_from_jdbc__anchored_features__feature_user_age'},
 {'name': 'feature_user_tax_rate',
  'id': 'd50c0db5-1fb7-4463-b00f-8e472a17bc7d',
  'qualifiedName': 'product_recommendation_from_jdbc__anchored_features__feature_user_tax_rate'},
 {'name': 'feature_user_gift_card_balance',
  'id': '2c12ca49-801f-40f8-ab6f-47e1de6b01af',
  'qualifiedName': 'product_recommendation_from_jdbc__anchored_features__feature_user_gift_card_balance'},
 {'name': 'feature_user_has_valid_credit_card',
  'id': 'b5dd2b16-9f15-4cf7-881e-29ef21efe731',
  'qualifiedName': 'product_recommendation_from_jdbc__anchored_features__feature_user_has_valid_credit_card'},
 {'name': 'feature_product_qu

In [5]:
feature_dict = client.get_features_from_registry(project_name=PROJECT_NAME, return_keys=True)

## we can list all features

In [6]:
[feat.name for feat in list(feature_dict[0].values())]

['feature_user_avg_purchase_for_90days',
 'feature_user_gift_card_balance',
 'feature_user_has_valid_credit_card',
 'feature_user_age',
 'feature_user_tax_rate',
 'feature_product_quantity',
 'feature_product_price',
 'feature_user_purchasing_power']

## we can list all type_key

In [7]:
[type_key.key_column for type_keys in list(feature_dict[1].values()) for type_key in type_keys]

['user_id',
 'user_id',
 'user_id',
 'user_id',
 'user_id',
 'product_id',
 'product_id',
 'user_id']

Let's use the helper function `get_result_df` to download the result and view it:

## 6. Feature Materialization

While Feathr can compute the feature value from the feature definition on-the-fly at request time, it can also pre-compute
and materialize the feature value to offline and/or online storage.

We can push the generated features to the online store like below:

In [8]:
os.environ['USERPROFILEDATA_USER'] = "postgres"
os.environ['USERPROFILEDATA_PASSWORD'] = "efcodd"
os.environ['USERPROFILEDATA_DRIVER'] = "org.postgresql.Driver"

os.environ['PRODUCTPROFILEDATA_USER'] = "postgres"
os.environ['PRODUCTPROFILEDATA_PASSWORD'] = "efcodd"
os.environ['PRODUCTPROFILEDATA_DRIVER'] = "org.postgresql.Driver"

os.environ['PURCHASE_HISTORY_DATA_USER'] = "postgres"
os.environ['PURCHASE_HISTORY_DATA_PASSWORD'] = "efcodd"
os.environ['PURCHASE_HISTORY_DATA_DRIVER'] = "org.postgresql.Driver"

In [9]:
os.environ['S3_ENDPOINT'] = "127.0.0.1:9000"
os.environ['S3_ACCESS_KEY'] = "zDaLfup26Qf2psWSVksV"
os.environ['S3_SECRET_KEY'] = "0uP2ohdTWeLXzV5UAlF8z5JVn17aEbr6OmOa9dkv"

In [10]:
# user key 

# user_feature_names = ['feature_user_avg_purchase_for_90days',
#        'feature_user_gift_card_balance', 'feature_user_has_valid_credit_card',
#        'feature_user_tax_rate', 'feature_user_age',
#        'feature_user_purchasing_power']
feature_dict = client.get_features_from_registry(project_name=PROJECT_NAME, return_keys=True)
key_user_id = feature_dict[1]["feature_user_avg_purchase_for_90days"][0]
user_feature_names = [fea for fea in feature_dict[1] if (feature_dict[1][fea][0].key_column == key_user_id.key_column) ]



In [11]:
# Materialize user features
# Note, you can only materialize features of same entity key into one table.
redisSink = RedisSink(table_name="user_features")
settings = MaterializationSettings(
    name="user_feature_setting",
    sinks=[redisSink],
    feature_names=user_feature_names,
)

client.materialize_features(settings=settings,
                            allow_materialize_non_agg_feature=True,
                            execution_configurations={
                              'spark.jars.plus.packages':"org.postgresql:postgresql:42.3.4"
                             }
                           )
client.wait_job_to_finish(timeout_sec=300)

2024-09-09 16:16:47.657 | WARNING  | feathr.utils._env_config_reader:get_from_env_or_akv:88 - Config KAFKA_SASL_JAAS_CONFIG is not found in the environment variable or the remote key value store.
2024-09-09 16:16:47.659 | INFO     | feathr.utils._env_config_reader:get:62 - Config monitoring__database__sql__url is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-09 16:16:47.660 | INFO     | feathr.utils._env_config_reader:get:62 - Config monitoring__database__sql__user is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-09 16:16:47.661 | WARNING  | feathr.utils._env_config_reader:get_from_env_or_akv:88 - Config MONITORING_DATABASE_SQL_PASSWORD is not found in the environment variable or the remote key value store.
2024-09-09 16:16:47.662 | WARNING  | feathr.spark_provider._localspark_submission:submit_feathr_job:78 - Loca

>

	found org.tukaani#xz;1.8 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.logging.log4j#log4j-core;2.17.2 in central
	found com.typesafe#config;1.3.4 in central
	found org.apache.hadoop#hadoop-mapreduce-client-core;3.3.2 in central
	found org.apache.hadoop#hadoop-yarn-client;3.3.2 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-guava;1.1.1 in central
	found commons-cli#commons-cli;1.2 in central
	found log4j#log4j;1.2.17 in central
	found org.eclipse.jetty.websocket#websocket-client;9.4.43.v20210629 in central
	found org.eclipse.jetty#jetty-client;9.4.43.v20210629 in central
	found org.eclipse.jetty#jetty-http;9.4.43.v20210629 in central
	found org.eclipse.jetty#jetty-util;9.4.43.v20210629 in central
	found org.eclipse.jetty#jetty-io;9.4.43.v20210629 in central
	found org.eclipse.jetty.websocket#websocket-common;9.4.43.v20210629 in central
	found org.eclipse.jetty.websocket#websocket-api;9.4.43.v20210629 in central
	found org.apache.hado

x

	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
	found org.codehaus.jackson#jackson-mapper-asl;1.9.13 in spark-list
	found org.codehaus.jackson#jackson-jaxrs;1.9.13 in central
	found org.codehaus.jackson#jackson-xc;1.9.13 in spark-list
	found com.sun.jersey.contribs#jersey-guice;1.19 in spark-list
	found com.sun.jersey#jersey-servlet;1.19 in central
	found com.fasterxml.jackson.module#jackson-module-jaxb-annotations;2.13.0 in central
	found jakarta.xml.bind#jakarta.xml.bind-api;2.3.3 in central
	found jakarta.activation#jakarta.activation-api;1.2.1 in central
	found com.fasterxml.jackson.jaxrs#jackson-jaxrs-json-provider;2.13.0 in central
	found com.fasterxml.jackson.jaxrs#jackson-jaxrs-base;2.13.0 in central
	found org.slf4j#slf4j-log4j12;1.7.30 in central
	found org.jline#jline;3.9.0 in central
	found io.netty#netty;3.10.6.Final in central
	found org.apache.hadoop#hadoop-common;3.3.2 in central
	found org.apache.commons#commons-math3;3.1.1 in central
	found commons-net

>

2024-09-09 16:17:21.733 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:234 - Spark job with pid 1034249 finished in: 34 seconds.


>

We can then get the features from the online store (Redis):

In [12]:
client.get_online_features(
    "user_features", "2", user_feature_names
)

[0.0, 204.0, True, 44, 0.03999999910593033, 304.0]

In [13]:
client.multi_get_online_features(
    "user_features", ["1", "2", "34", "45", "78"], user_feature_names
)

{'1': [0.0, 1515.0, True, 23, 0.029999999329447746, 1615.0],
 '2': [0.0, 204.0, True, 44, 0.03999999910593033, 304.0],
 '34': [0.0, 1939.0, True, 56, 0.019999999552965164, 2039.0],
 '45': [0.0, 1479.0, True, 37, 0.03999999910593033, 1579.0],
 '78': [0.0, 1146.0, True, 45, 0.05000000074505806, 1246.0]}

In [14]:
# product key

# product_feature_names = [
#        'feature_product_price', 'feature_product_quantity',
# ]
feature_dict = client.get_features_from_registry(project_name=PROJECT_NAME, return_keys=True)
key_product_id = feature_dict[1]["feature_product_quantity"][0]
product_feature_names = [fea for fea in feature_dict[1] if (feature_dict[1][fea][0].key_column == key_product_id.key_column) ]

In [16]:
# Materialize user features
# Note, you can only materialize features of same entity key into one table.
product_redisSink = RedisSink(table_name="product_features1")
product_settings = MaterializationSettings(
    name="product_feature_setting",
    sinks=[product_redisSink],
    feature_names=product_feature_names,
)

client.materialize_features(settings=product_settings,
                            allow_materialize_non_agg_feature=True,
                            execution_configurations={ 
                              'spark.jars.plus.packages':"org.postgresql:postgresql:42.3.4"
                            }
                           )
client.wait_job_to_finish(timeout_sec=300)

2024-09-09 16:20:50.864 | WARNING  | feathr.utils._env_config_reader:get_from_env_or_akv:88 - Config KAFKA_SASL_JAAS_CONFIG is not found in the environment variable or the remote key value store.
2024-09-09 16:20:50.866 | INFO     | feathr.utils._env_config_reader:get:62 - Config monitoring__database__sql__url is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-09 16:20:50.867 | INFO     | feathr.utils._env_config_reader:get:62 - Config monitoring__database__sql__user is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-09 16:20:50.867 | WARNING  | feathr.utils._env_config_reader:get_from_env_or_akv:88 - Config MONITORING_DATABASE_SQL_PASSWORD is not found in the environment variable or the remote key value store.
2024-09-09 16:20:50.868 | WARNING  | feathr.spark_provider._localspark_submission:submit_feathr_job:78 - Loca

>

	found org.apache.logging.log4j#log4j-core;2.17.2 in central
	found com.typesafe#config;1.3.4 in central
	found org.apache.hadoop#hadoop-mapreduce-client-core;3.3.2 in central
	found org.apache.hadoop#hadoop-yarn-client;3.3.2 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-guava;1.1.1 in central
	found commons-cli#commons-cli;1.2 in central
	found log4j#log4j;1.2.17 in central
	found org.eclipse.jetty.websocket#websocket-client;9.4.43.v20210629 in central
	found org.eclipse.jetty#jetty-client;9.4.43.v20210629 in central
	found org.eclipse.jetty#jetty-http;9.4.43.v20210629 in central
	found org.eclipse.jetty#jetty-util;9.4.43.v20210629 in central
	found org.eclipse.jetty#jetty-io;9.4.43.v20210629 in central
	found org.eclipse.jetty.websocket#websocket-common;9.4.43.v20210629 in central
	found org.eclipse.jetty.websocket#websocket-api;9.4.43.v20210629 in central
	found org.apache.hadoop#hadoop-annotations;3.3.2 in central
	found org.apache.hadoop#hadoop-yarn-api;3.3.2 in cen

x

	found org.codehaus.jackson#jackson-jaxrs;1.9.13 in central
	found org.codehaus.jackson#jackson-xc;1.9.13 in spark-list
	found com.sun.jersey.contribs#jersey-guice;1.19 in spark-list
	found com.sun.jersey#jersey-servlet;1.19 in central
	found com.fasterxml.jackson.module#jackson-module-jaxb-annotations;2.13.0 in central
	found jakarta.xml.bind#jakarta.xml.bind-api;2.3.3 in central
	found jakarta.activation#jakarta.activation-api;1.2.1 in central
	found com.fasterxml.jackson.jaxrs#jackson-jaxrs-json-provider;2.13.0 in central
	found com.fasterxml.jackson.jaxrs#jackson-jaxrs-base;2.13.0 in central
	found org.slf4j#slf4j-log4j12;1.7.30 in central
	found org.jline#jline;3.9.0 in central
	found io.netty#netty;3.10.6.Final in central
	found org.apache.hadoop#hadoop-common;3.3.2 in central
	found org.apache.commons#commons-math3;3.1.1 in central
	found commons-net#commons-net;3.6 in central
	found commons-collections#commons-collections;3.2.2 in central
	found org.eclipse.jetty#jetty-server;9

>

2024-09-09 16:21:24.920 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:234 - Spark job with pid 1037568 finished in: 34 seconds.


>

In [17]:
product_feature_names

['feature_product_quantity', 'feature_product_price']

In [18]:
client.get_online_features(
    "product_features1", "1", product_feature_names
)

[14.0, 104.39189147949219]

In [19]:
client.multi_get_online_features(
    "product_features1", [str(i) for i in range(1,100,1)], product_feature_names
)

{'1': [14.0, 104.39189147949219],
 '2': [69.0, 60.08137893676758],
 '3': [16.0, 24.84902000427246],
 '4': [10.0, 152.512939453125],
 '5': [88.0, 1.3630683422088623],
 '6': [49.0, 67.41737365722656],
 '7': [88.0, 33.198917388916016],
 '8': [61.0, 183.12388610839844],
 '9': [31.0, 8.413022994995117],
 '10': [88.0, 59.487388610839844],
 '11': [13.0, 50.92708206176758],
 '12': [40.0, 80.02034759521484],
 '13': [55.0, 136.61766052246094],
 '14': [70.0, 87.16132354736328],
 '15': [97.0, 11.105215072631836],
 '16': [11.0, 135.34413146972656],
 '17': [11.0, 100.65690612792969],
 '18': [83.0, 39.408809661865234],
 '19': [28.0, 135.954833984375],
 '20': [63.0, 138.9633331298828],
 '21': [34.0, 44.26852798461914],
 '22': [57.0, 22.263668060302734],
 '23': [80.0, 180.9148712158203],
 '24': [54.0, 101.99103546142578],
 '25': [6.0, 68.22028350830078],
 '26': [59.0, 85.59817504882812],
 '27': [70.0, 81.89146423339844],
 '28': [85.0, 130.9653778076172],
 '29': [93.0, 93.73111724853516],
 '30': [62.0, 

# get features for infer step

In [20]:
infer_user_features = client.multi_get_online_features(
    "user_features", ["1", "2", "34", "45", "78"], user_feature_names
)
infer_user_features_df = pd.DataFrame(infer_user_features).T.reset_index()
infer_user_features_df.columns =["user_id"] + user_feature_names

In [21]:
infer_product_features = client.multi_get_online_features(
    "product_features1", [str(i) for i in range(1,100,1)], product_feature_names
)
infer_product_features_df = pd.DataFrame(infer_product_features).T.reset_index()
infer_product_features_df.columns =["product_id"] + product_feature_names

In [22]:
infer_user_features_df["cross_key"] = 1
infer_product_features_df["cross_key"] = 1

In [23]:
features_df = pd.merge(infer_user_features_df, infer_product_features_df, on ='cross_key').drop(labels="cross_key", axis=1)

In [24]:
order_features = ['user_id', 'product_id', 'feature_user_avg_purchase_for_90days',
       'feature_product_price', 'feature_product_quantity',
       'feature_user_gift_card_balance', 'feature_user_has_valid_credit_card',
       'feature_user_tax_rate', 'feature_user_age',
       'feature_user_purchasing_power']

In [25]:
features_df[order_features]

,user_id,product_id,feature_user_avg_purchase_for_90days,feature_product_price,feature_product_quantity,feature_user_gift_card_balance,feature_user_has_valid_credit_card,feature_user_tax_rate,feature_user_age,feature_user_purchasing_power
0,1,1,0.0,104.391891,14.0,1515.0,True,0.03,23,1615.0
1,1,2,0.0,60.081379,69.0,1515.0,True,0.03,23,1615.0
2,1,3,0.0,24.849020,16.0,1515.0,True,0.03,23,1615.0
3,1,4,0.0,152.512939,10.0,1515.0,True,0.03,23,1615.0
4,1,5,0.0,1.363068,88.0,1515.0,True,0.03,23,1615.0
...,...,...,...,...,...,...,...,...,...,...
490,78,95,0.0,10.849150,91.0,1146.0,True,0.05,45,1246.0
491,78,96,0.0,16.686001,97.0,1146.0,True,0.05,45,1246.0
492,78,97,0.0,108.277031,35.0,1146.0,True,0.05,45,1246.0
493,78,98,0.0,179.862762,61.0,1146.0,True,0.05,45,1246.0
